# Classification of wind gust events

This notebook enables classification of daily maximum wind gust events into specific classes of storm type. Following the approach of Cook (2023)[1], the notebook will plot the time history of gust wind speed, mean wind speed, direction, temperature and station pressure over a 2-hour window around each gust event. The user then selects the storm type from one of 8 options:

* Unclassified
* Spike
* Synoptic storm
* Synoptic front
* Storm burst
* Front down
* Front up
* Thunderstorm

The definitions of these events largely follow Cook's (2023) definitions, but we have added a "Synoptic front" event, which are typically events such as southerly busters common along the NSW coast, or similar gust events that have characteristics of a density current flow. "Synoptic storm" will also capture tropical cyclone events in northern parts of the country, but these can also be identified by comparing to the location of TC events at the time of a peak gust.

I suggest running this notebook via a browser rather than directly in VS Code. In a command prompt, run the following:

```
conda activate process
cd <path\to\this\folder>
jupyter notebook
```

This will start the notebook server, and open the folder listing in the default browser. You can then open this notebook and run the code blocks.


[1]: Cook, N. J., 2023: Automated classification of gust events in the contiguous USA. *Journal of Wind Engineering and Industrial Aerodynamics*, **234**, 105330, https://doi.org/10.1016/j.jweia.2023.105330.


In [11]:
%matplotlib inline

import os
import time
from functools import partial

import pandas as pd
import numpy as np
from datetime import datetime, timedelta

import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import Layout, Box, VBox, HBox
from IPython.display import display, clear_output
import seaborn as sns
sns.set_style('whitegrid')
import warnings
warnings.filterwarnings("ignore", 'SettingWithCopyWarning')

In [12]:
BASEDIR = r"X:\georisk\HaRIA_B_Wind\data\derived\obs\1-minute\events"
hqstations = pd.read_csv(os.path.join(BASEDIR, "hqstations.csv"), index_col="stnNum")
options = [(stnName, idx) for idx, stnName in hqstations['stnName'].items()]
out = widgets.Output()

In [13]:
def loadData(stnNum):
    fname = os.path.join(BASEDIR, "events", f"{stnNum:06d}.pkl")
    df = pd.read_pickle(fname)
    df.reset_index(inplace=True)
    df.set_index(['date'], inplace=True)
    return df

def plotEvent(pdf, fig):
    fig.clear()
    ax = fig.add_axes([0, 0, 1, 1])
    ax2 = ax.twinx()
    axt = ax.twinx()
    axp = ax.twinx()
    axr = ax.twinx()
    
    pdf.plot.line(x='tdiff', y='windgust', marker='o', ax=ax)
    pdf.plot.line(x='tdiff', y='windspd', marker='o', ax=ax, color='0.5')
    pdf.plot.scatter(x='tdiff', y='winddir', ax=ax2, color='g', marker='o',)
    pdf.plot.line(x='tdiff', y='tempanom', color='r', ax=axt)
    pdf.plot.line(x='tdiff', y='dpanom', color='orangered', ax=axt)
    pdf.plot.line(x='tdiff', y='stnpanom', color='purple', ax=axp)
    pdf.plot.line(x='tdiff', y='rh', color='lime', ax=axr)
    
    axt.spines['right'].set_position(("axes", 1.075))
    axt.spines[['right']].set_color('r')
    axt.yaxis.label.set_color('r')
    axt.tick_params(axis='y', colors='r')
    axt.set_ylabel(r"Temperature/dewpoint anomaly [$^o$C]")
    
    ax.set_ylabel("Gust wind speed [km/h]")
    
    axp.spines[['right']].set_position(('axes', 1.25))
    axp.spines[['right']].set_color('purple')
    axp.yaxis.label.set_color('purple')
    axp.tick_params(axis='y', colors='purple')
    axp.set_ylabel("Pressure anomaly [hPa]")
    
    axr.spines[['right']].set_position(('axes', 1.15))
    axr.spines[['right']].set_color('lime')
    axr.yaxis.label.set_color('lime')
    axr.tick_params(axis='y', colors='lime')
    axr.set_ylabel("Relative humidity [%]")
    
    gmin, gmax = ax.get_ylim()
    pmin, pmax = axp.get_ylim()
    tmin, tmax = axt.get_ylim()
    ax.set_ylim((0, max(gmax, 100)))
    axp.set_ylim((min(-2.0, pmin), max(pmax, 2.0)))
    axt.set_ylim((min(-2.0, tmin), max(tmax, 2.0)))
    ax2.set_ylim((0, 360))
    ax2.set_yticks(np.arange(0, 361, 90))
    axr.set_ylim((0, 100))
    ax.set_title(pdf.index[0])
    ax.grid(True)
    ax2.grid(False)
    axt.grid(False)
    axp.grid(False)
    axr.grid(False)
    return fig

class Plotter:
    def __init__(self, df, fig, initial=0):
        clear_output(wait=False)
        self.df = df
        self.fig = fig
        self.value = initial
        self.nevents = len(self.df.index.unique('date'))
        eventdate = self.df.index.unique('date')[self.value]
        plotEvent(self.df[self.df.index==eventdate], self.fig)
        
    def plot(self, amount=1):
        with out:
            if self.value < self.nevents - 1:
                self.value += amount
                print(f"Event #: {self.value} of {self.nevents}")
                eventdate = self.df.index.unique('date')[self.value]
                plotEvent(self.df[self.df.index==eventdate], self.fig)
                return self.fig
            else:
                #print("No more events")
                self.fig.clf()
                return None

    def __iter__(self, sentinal=False):
        return iter(self.plot, sentinal)

    def __del__(self):
        print("deleting plotter")
        plt.close(self.fig)



def button_callback(w):
    #out.clear_output(wait=True)
    with out:
        eventdate = df.index.unique('date')[plotter.value]
        print(f"Date: {eventdate} | Storm type: {selector.value}")
        eventlist.append((stnselector.value, eventdate, selector.value))

#@out.capture(clear_output=True)
def plotnext(plotter, w):
    #print(dir(w))
    out.clear_output(wait=False)
    with out:
        plotter.plot()
        if plotter.value <= plotter.nevents - 1:
            display(plotter.fig)
        else:
            del plotter


In [14]:
def dropdown_eventhandler(change):
    out.clear_output(wait=False)
#    global df
#    df = loadData(change.new)
stnselector = widgets.Dropdown(options=options, description='Station', value=3003)

In [15]:
selector = widgets.ToggleButtons(
    options=["Unclassified", "Spike", "Synoptic storm", "Synoptic front",
             "Storm-burst", "Front down", "Front up", "Thunderstorm"],
    description="Storm type",
    disabled=False, 
    layout=Layout(width='auto'),
    style={"button_width": "100px"}
)
assignbtn = widgets.Button(description="Assign storm type")
nextevent = widgets.Button(description="Next event")

eventlist = []


We generate a dropdown list of stations to select from. Arguably, one does not have to go through all stations, but to not do that will reduce the training sample of events. 

In [16]:
stnselector.observe(dropdown_eventhandler, names='value')
display(stnselector)


Dropdown(description='Station', options=(('BROOME AIRPORT', 3003), ('PORT HEDLAND AIRPORT', 4032), ('LEARMONTH…

After selecting a station, run the next cell to generate the plot of the first event for that station, as well as show the classification buttons. 

**Each time a new station is selected, you need to re-run the next code cell**

In [17]:
out = widgets.Output()
df = None
df = loadData(stnselector.value)
nevents = len(df.index.unique('date'))
fig = plt.figure(figsize=(12, 6))
plotter = Plotter(df, fig)
assignbtn.on_click(button_callback)
nextevent.on_click(partial(plotnext, plotter))

display(selector, HBox([assignbtn, nextevent]))
out.clear_output(wait=True)
with out:
    plt.show()

out



ToggleButtons(description='Storm type', layout=Layout(width='auto'), options=('Unclassified', 'Spike', 'Synopt…

Output()

Now we can show the list of storm types, keyed by station number and date. Then save the dataframe to a file for further analysis using the `to_csv` method on the dataframe.

In [ ]:
pd.DataFrame(eventlist, columns=['stnNum', 'date', 'stormType']).to_csv(r"X:\georisk\HaRIA_B_Wind\data\derived\obs\1-minute\events\CA_20230518_Yarrawonga.csv")